# 4_2_Clustering_SE.ipynb
Further sample training and test data of explanatory variables after sampling in [4_1_Feature_extraction_SE.ipynb] in SE.

### input
- 4_Feature_extraction_nonPCA/output/Target_sub_cluster.csv : A file that the paths were sub-clustered based on Targets. In this notebook, we only used the column 'HSA','sub_ID' and 'down_index_new_0' in this file.
- 9_Integration_SE_TI_Target_datafile/Y_binary_SE.npz : A file with Path ID and SE linked.
- 5_X_train_test_datafile/X_Feature_Extraction/train/X_train_SE_*.npz : Training data for explanatory variables after sampling in SE.
- 5_X_train_test_datafile/X_Feature_Extraction/test/X_test_SE_*.npz : Test data for explanatory variables after sampling in SE.

### output
- 5_X_train_test_datafile_nonPCA/train/X_train_Clustering_SE_*.npz : Training data for explanatory variables after sampling for the second time in SE.
- 5_X_train_test_datafile_nonPCA/test/X_test_Clustering_SE_*.npz : Test data for explanatory variables after sampling for the second time in SE.
- 4_Feature_extraction_nonPCA/output/SE_Feature_Cluster_ID.csv : A file which contains the number of features of explanatory variables after sampling.

In [1]:
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse import save_npz, load_npz
import math

In [2]:
y = pd.DataFrame(load_npz('../9_Integration_SE_TI_Target_datafile/Y_binary_SE.npz').toarray())

In [3]:
df_t = pd.read_csv('output/Target_sub_cluster.csv',header = 0, index_col=0).drop(columns = 'index')

In [4]:
df_cluster = pd.DataFrame()
for i in range(len(y.columns)):
    X_train_m = load_npz('../5_X_train_test_datafile/X_Feature_Extraction/train/X_train_SE_'+ str(i) +'.npz').toarray()
    
    X_test_m = load_npz('../5_X_train_test_datafile/X_Feature_Extraction/test/X_test_SE_'+ str(i) +'.npz').toarray()
    
    y_index = pd.DataFrame(y[y[i]==1].index).rename(columns = {0:'ID'})
    y_index['ID_sub'] = y_index['ID'].rank(method='dense') - 1
    
    df2 = pd.merge(y_index, df_t, left_on = 'ID', right_on = 'down_index_new_0').drop(columns = 'down_index_new_0')
    sub_n = len(df2[['HSA', 'sub_ID']].drop_duplicates())
    df1 = df2.groupby(['sub_ID', 'HSA']).apply(lambda x: x.sample(n=min(math.ceil(500/sub_n), len(x)), random_state=0)).reset_index(level=['HSA','sub_ID'], drop=True).sort_values('ID_sub')
    train = pd.DataFrame(X_train_m)[list(df1['ID_sub'])]
    test = pd.DataFrame(X_test_m)[list(df1['ID_sub'])]
    save_npz('../5_X_train_test_datafile_nonPCA/train/X_train_Clustering_SE_'+ str(i) +'.npz', csr_matrix(train))
    save_npz('../5_X_train_test_datafile_nonPCA/test/X_test_Clustering_SE_'+ str(i) +'.npz', csr_matrix(test))
    df1['SE_ID'] = i
    df1['SE_count'] = len(df1.dropna())
    df_cluster = pd.concat([df_cluster, df1])
        
    df_cluster = df_cluster.reset_index(drop = True)

In [5]:
df_cluster

,ID,ID_sub,sub_ID,HSA,SE_ID,SE_count
0,33,0.0,2,"[1003, 3791]",0,461
1,34,1.0,3,"[1003, 3791]",0,461
2,35,2.0,4,"[1003, 3791]",0,461
3,36,3.0,5,"[1003, 3791]",0,461
4,43,10.0,1,"[1003, 3791]",0,461
...,...,...,...,...,...,...
52932,66087,726.0,1,[7298],176,91
52933,66088,727.0,1,[7298],176,91
52934,66106,745.0,1,[7298],176,91
52935,66150,789.0,1,[7298],176,91


In [6]:
df_cluster.to_csv('output/SE_Feature_Cluster_ID.csv')